<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula9c_mnist_flat_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
train_flat_images = train_images.reshape((60000, 28 * 28))
test_flat_images = test_images.reshape((10000, 28 * 28))
train_flat_images.shape

(60000, 784)

In [3]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

train_flat_images_tensor = torch.tensor(train_flat_images, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
train_dataset = TensorDataset(train_flat_images_tensor, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_flat_images_tensor = torch.tensor(test_flat_images, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

class BasicTorchNN(nn.Module):
  def __init__(self, num_classes):
    super(BasicTorchNN, self).__init__()
    self.fc1 = nn.Linear(28 * 28, 512)
    self.fc2 = nn.Linear(512, num_classes)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = BasicTorchNN(10)
model.to(device)

optimizer = optim.RMSprop(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

epochs = 20
batch_size = 128

for epoch in range(epochs):
  for batch in train_loader:
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

criterion = nn.CrossEntropyLoss()
with torch.no_grad():
  test_flat_images_tensor = test_flat_images_tensor.to(device)
  test_labels_tensor = test_labels_tensor.to(device)
  outputs = model(test_flat_images_tensor)
  loss = criterion(outputs, test_labels_tensor)
  print(f"Test Loss: {loss.item()}")
  _, predicted = torch.max(outputs.data, 1)
  accuracy = (predicted == test_labels_tensor).sum().item() / len(test_labels_tensor)
  print(f"Accuracy: {accuracy}")


Epoch 1/20, Loss: 0.4410780966281891
Epoch 2/20, Loss: 0.3133017122745514
Epoch 3/20, Loss: 0.367511510848999
Epoch 4/20, Loss: 0.11978668719530106
Epoch 5/20, Loss: 0.1603698879480362
Epoch 6/20, Loss: 0.032896194607019424
Epoch 7/20, Loss: 0.06642156094312668
Epoch 8/20, Loss: 0.0002354238386033103
Epoch 9/20, Loss: 0.0016452261479571462
Epoch 10/20, Loss: 0.09129790216684341
Epoch 11/20, Loss: 0.013662546873092651
Epoch 12/20, Loss: 0.005115180741995573
Epoch 13/20, Loss: 0.00021203015057835728
Epoch 14/20, Loss: 0.03095310926437378
Epoch 15/20, Loss: 0.0005734392325393856
Epoch 16/20, Loss: 0.0035133296623826027
Epoch 17/20, Loss: 0.0001009546613204293
Epoch 18/20, Loss: 2.0206631234032102e-05
Epoch 19/20, Loss: 0.002088273176923394
Epoch 20/20, Loss: 0.002048423048108816
Test Loss: 0.21316586434841156
Accuracy: 0.9751


In [13]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from sklearn.metrics import accuracy_score

class TorchWrappedNN(BaseEstimator, ClassifierMixin):
  def __init__(self, epochs=5, batch_size=128, model_fabric=BasicTorchNN):
    self.epochs = epochs
    self.batch_size = batch_size
    self.model_fabric = model_fabric
    self.verbose = True

  def fit(self, X, y):
    self.labels, ids = np.unique(y, return_inverse=True)
    ytensor = torch.tensor(ids, dtype=torch.long)
    xtensor = torch.tensor(X, dtype=torch.float32)
    dataset = TensorDataset(xtensor, ytensor)
    loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
    self.model = self.model_fabric(len(self.labels))
    self.model.to(device)
    optimizer = optim.RMSprop(self.model.parameters(), lr=0.0001)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(self.epochs):
      for batch in loader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = self.model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      if self.verbose:
        print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss.item()}")
    return self
  def predict(self, X):
    with torch.no_grad():
      xtensor = torch.tensor(X, dtype=torch.float32)
      xtensor = xtensor.to(device)
      outputs = self.model(xtensor)
      _, predicted = torch.max(outputs.data, 1)
      return self.labels[predicted.cpu().numpy()]

modelo = TorchWrappedNN()
modelo.fit(train_flat_images, train_labels)
ypred = modelo.predict(test_flat_images)
accuracy_score(test_labels, ypred)

Epoch 1/5, Loss: 0.14165109395980835
Epoch 2/5, Loss: 0.058302175253629684
Epoch 3/5, Loss: 0.19886456429958344
Epoch 4/5, Loss: 0.20076513290405273
Epoch 5/5, Loss: 0.054836858063936234


0.9649

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("modelo", TorchWrappedNN())
])

pipeline.fit(train_flat_images, train_labels)
ypred = pipeline.predict(test_flat_images)
accuracy_score(test_labels, ypred)

Epoch 1/5, Loss: 0.44596007466316223
Epoch 2/5, Loss: 0.2545887529850006
Epoch 3/5, Loss: 0.17768794298171997
Epoch 4/5, Loss: 0.16397058963775635
Epoch 5/5, Loss: 0.15477736294269562


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


0.9418

In [15]:
from sklearn.base import TransformerMixin

class Divide255(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X / 255.0

pipeline = Pipeline([
    ("scaler", Divide255()),
    ("modelo", TorchWrappedNN())
])

pipeline.fit(train_flat_images, train_labels)
ypred = pipeline.predict(test_flat_images)
accuracy_score(test_labels, ypred)

Epoch 1/5, Loss: 0.2825833857059479
Epoch 2/5, Loss: 0.23002640902996063
Epoch 3/5, Loss: 0.2644854187965393
Epoch 4/5, Loss: 0.2588998079299927
Epoch 5/5, Loss: 0.12089911103248596


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


0.9445

In [16]:
class Shape2Flat(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X.reshape((-1, 28 * 28))

pipeline = Pipeline([
    ("scaler", Divide255()),
    ("shape2flat", Shape2Flat()),
    ("modelo", TorchWrappedNN(20))
])

pipeline.fit(train_images, train_labels)
ypred = pipeline.predict(test_images)
accuracy_score(test_labels, ypred)

Epoch 1/20, Loss: 0.45673128962516785
Epoch 2/20, Loss: 0.33961692452430725
Epoch 3/20, Loss: 0.2299308180809021
Epoch 4/20, Loss: 0.24339406192302704
Epoch 5/20, Loss: 0.1098884716629982
Epoch 6/20, Loss: 0.09645608067512512
Epoch 7/20, Loss: 0.1897345781326294
Epoch 8/20, Loss: 0.052187979221343994
Epoch 9/20, Loss: 0.14893117547035217
Epoch 10/20, Loss: 0.1519756019115448
Epoch 11/20, Loss: 0.09983271360397339
Epoch 12/20, Loss: 0.06671654433012009
Epoch 13/20, Loss: 0.11783039569854736
Epoch 14/20, Loss: 0.06224215030670166
Epoch 15/20, Loss: 0.050026800483465195
Epoch 16/20, Loss: 0.020663077011704445
Epoch 17/20, Loss: 0.09875998646020889
Epoch 18/20, Loss: 0.08698626607656479
Epoch 19/20, Loss: 0.07865362614393234
Epoch 20/20, Loss: 0.13838432729244232


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


0.974